# Approach 


- Get lyrics from 5 or 6 rappers using the Genius API
- Use the Datamuse API to get words that rhyme with a ample of words from those lyrics
- Get words that don't rhyme with those words also
- Make a dataframe of wordA | wordB | rhyme
- Train model on jupyter collab

In [2]:
import os
import re
import time
import requests
import itertools
import pandas as pd
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
load_dotenv()

False

# Functions 

In [5]:
def get_rhymes(word):
    
    url = f"https://api.datamuse.com/words?rel_rhy={word.lower()}"
    
    response = requests.get(url)
    
    return response.json()

# Iterate thorugh each word and get all the rhymes for each word 

To do this we will look up each word and get all the rhymes using the datamuse API. We will then create a row for each rhyme combination an assign them a rhyme group ID.

In [38]:
rhyme_data = []
rhyme_id = 1
rhyme_group_id = 1
with open("en.dict","r") as dict_file:
    while True:
        file_contents=dict_file.readline().strip()
        if file_contents=="":
            break
        rhyme_response = get_rhymes(file_contents)
        if len(rhyme_response) > 0:
            # list all words returned by the response
            rhyming_words = [rhyme['word'] for rhyme in rhyme_response] + [file_contents]
            all_rhyme_combinations = list(itertools.combinations(rhyming_words, 2))
            # create an entry for all possible rhyme pairs returned
            for rhyme_pair in all_rhyme_combinations:
                rhyme_data.append(
                    {
                            'rhyme_id': rhyme_id,
                            'rhyme_group_id': rhyme_group_id,
                            'word_a': rhyme_pair[0],
                            'word_b': rhyme_pair[1],
                            'rhyme': 1
                    }
                )
                rhyme_id+=1
            rhyme_group_id+=1
        # lets just save the dataframe every 500 words
        if rhyme_group_id % 10 == 0 and rhyme_group_id != 0:
            rhyme_df = pd.DataFrame(rhyme_data)
            rhyme_df.to_pickle('data/rhymes/rhyme_df.pkl')

# convert to dataframe
rhyme_df = pd.DataFrame(rhyme_data)
rhyme_df = rhyme_df.drop_duplicates(subset=['word_a', 'word_b'], keep='first')
rhyme_df.to_csv('data/rhymes/rhyme_df.csv', index=False)

KeyboardInterrupt: 

In [40]:
rhyme_df=pd.read_pickle("data/rhymes/rhyme_df.pkl")
rhyme_df = pd.DataFrame(rhyme_data)
rhyme_df = rhyme_df.drop_duplicates(subset=['word_a', 'word_b'], keep='first')
rhyme_df.to_csv('data/rhymes/rhyme_df.csv', index=False)

In [5]:
rhyme_df = pd.read_csv('data/rhymes/rhyme_df.csv')
rhyme_df.loc[rhyme_df['rhyme_group_id']==261].sample(5).reset_index(drop=True)

,rhyme_id,rhyme_group_id,word_a,word_b,rhyme
0,627022,261,ploughing,somehow hung,1
1,627024,261,ploughing,allowing,1
2,627063,261,allow ing,bow hung,1
3,627034,261,soughing,thao hung,1
4,627068,261,allow ing,thao hung,1


# Now create a dataframe of negative samples (i.e. words that don't rhyme) 

To do this we will basically iterate through each rhyme group ID, select a sample of words from another rhyme group (that shouldn't rhyme) and assign those words to the `word_b`column

In [7]:
non_rhyme_df = rhyme_df.copy()
for rhyme_group in list(rhyme_df['rhyme_group_id'].drop_duplicates()):
    
    words_in_group = len(rhyme_df.loc[rhyme_df['rhyme_group_id'] == rhyme_group])
    
    other_rhyme_samples = list(
        non_rhyme_df.loc[non_rhyme_df['rhyme_group_id'] != rhyme_group, 'word_b'].sample(words_in_group)
    )
    
    non_rhyme_df.loc[non_rhyme_df['rhyme_group_id'] == rhyme_group, 'word_b'] = other_rhyme_samples
    
non_rhyme_df['rhyme'] = 0
non_rhyme_df = non_rhyme_df.drop_duplicates(subset=['word_a', 'word_b'], keep='first')
non_rhyme_df.to_csv('data/rhymes/non_rhyme_df.csv', index=False)
print('Done!')

Done!
